In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

In [19]:
llm = ChatOpenAI(
    temperature = 0.1
)


splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)


# txt,doc,pdf.html,excel 등등 다양한 file들을 load 할 수 있다.
loader = UnstructuredFileLoader("../KAIA-JKIICE.pdf")

docs = loader.load_and_split(text_splitter=splitter)


cache_dir = LocalFileStore("./.cache/")

# Embedding은 text에 의미별로 적절한 점수를 부여해서 vector 형식으로 표현한것,

embeddings = OpenAIEmbeddings()

# ChacheBackedEmbedding을 사용하여 만들어진 Embedding을 cache(저장) 한다.

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings,cache_dir)

# Vector store를 호출 해주고 FAISS사용
# from_documents 메서드는 document 별로 embedding 작업 후 결과를 저장한 vector store를 반환
# 이를 이용해 document 검색도 하고, 연관성이 높은 document들을 찾는다.
vectorstore = FAISS.from_documents(docs,cached_embeddings)

# retriver는 document들의 list를 반환
retriever = vectorstore.as_retriever()


map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see 
            if any of the text is relevant to answer the question. 
            Return any relevant text verbatim. 
            If there is no relevant text, return : ''
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt|llm

# 이 함수는 retriever에서 반한한 값을 input으로 입력받는 함수
def map_docs(inputs):
    # documents = inputs["documents"]
    # question = inputs["question"]


    # results = []
    # for document in documents:
    #     result = map_doc_chain.invoke({
    #         "context": document.page_content,
    #         "question":question
    #     }).content
    #     results.append(result)
    # print(results)

    # # chain에서 context에 넣을 려면 리스트 형태가 아니라 string으로 만들어주기 위해
    # # 줄바꿈으로 구분
    # results="\n\n".join(results)
    # return results

    documents = inputs["documents"]
    question = inputs["question"]
    return "\n\n".join(
        map_doc_chain.invoke(
            {"context" : doc.page_content,"question":question}
        ).content
        for doc in documents
    )



# map_chain은 마지막 chain 내부에서 호출
# map_chain은 document가 필요하고 document는 retriever를 사용해서 얻을 수 있다.
# 그리고 사용자의 질문 내용을 알아야한다. 그래야만 LLM에게 요청할 수 있다.
# 각 document를 살표보면서 사용자 질문에 대답하는데 필요한 정보가 담겨있는지 알아봐달라고 하면된다.
map_chain = {"documents" : retriever,"question":RunnablePassthrough()}|RunnableLambda(map_docs)


final_prompt = ChatPromptTemplate.from_messages([
    ("system",
    """
Given the following extracted parts of a long document and a question, create a final answer.
    If you don't know the answer, just say that you don't know. Don't try 
    to make up an answer.
     -----
     {context}
"""
    ),
     ("human","{question}")
])

# RunnablePassthrough 말그대로 통과게 해준다.("이 논문은 무엇을 하는 논문인지 설명해주고, 한글로 해석해서 알려줘." 
# 이 입력값이 RunnablePassthrough를 통해서 question에 전달된다.)

chain = {"context":map_chain,"question":RunnablePassthrough()}|final_prompt|llm


chain.invoke("이 논문은 무엇을 하는 논문인지 설명해주고, 한글로 해석해서 알려줘.")

Created a chunk of size 1029, which is longer than the specified 600
Created a chunk of size 864, which is longer than the specified 600
Created a chunk of size 643, which is longer than the specified 600
Created a chunk of size 636, which is longer than the specified 600
Created a chunk of size 726, which is longer than the specified 600
Created a chunk of size 918, which is longer than the specified 600
Created a chunk of size 692, which is longer than the specified 600
Created a chunk of size 1431, which is longer than the specified 600


AIMessage(content='이 논문은 열화상 카메라와 광각 카메라를 동시에 적용하여 YOLO 기반의 딥러닝 알고리즘을 이용한 데이터 분석을 기반으로 도로의 블랙아이스, 포트홀 및 장애물을 실시간으로 탐지하며, 도로 사용자에게 실시간으로 중요한 안전 정보를 제공하는 실용적이고 효율적인 도로 안전 시스템을 개발하는 것이다.\n\n이 논문은 도로 안전 시스템을 개발하기 위해 열화상 카메라와 광각 카메라를 함께 사용하여 딥러닝 알고리즘을 적용하고, 도로 위의 블랙아이스, 포트홀 및 장애물을 실시간으로 감지하여 도로 이용자에게 중요한 안전 정보를 제공하는 것을 목표로 한다.')